In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough

In [ ]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Jon_Jones")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"Número de chunks: {len(chunks)}")

In [ ]:
# Modelos
embeddings = OllamaEmbeddings(model="nomic-embed-text")
llm = ChatOllama(model="llama3.2", temperature=0.001)

In [ ]:
# Persiste vetores no banco de dados local
Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./data",
)

In [ ]:
vectorstore = Chroma(persist_directory="./data", embedding_function=embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
# Testando o retriever
retriever.invoke("Where was Jon Jones born?")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful AI assistant that answers the user's question based only on the context below:
            <context>
                {context}
            </context>

            Answer the user's questions considering the above context only.
            """,
        ),
        ("human", "{question}"),
    ]
)

prompt

In [ ]:
chain = (
    {"question": RunnablePassthrough(), "context": retriever} | prompt | llm | StrOutputParser()
)

In [ ]:
stream = chain.stream("Did Jon Jones win the fight against Miotic?")

for chunk in stream:
    print(chunk, end="", flush=True)
print()

In [ ]:
def prompt_printer(prompt):
    print(prompt)

    return prompt

# Debuggin chain
chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt
    | prompt_printer
    | llm
    | StrOutputParser()
)

answer = chain.invoke("Did Jon Jones win the fight against Miotic?")
print(answer)

In [ ]:
prompt2 = """You are a text translator. Everything the user says, you translate into english.
"""

prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", prompt2),
    ("human", "{question}")
])

prompt3 = """You are a text translator. Everything the user says, you translate into portuguese.
"""

prompt_template3 = ChatPromptTemplate.from_messages(
    [("system", prompt3), ("human", "{question}")]
)

In [ ]:
chain_translate_input = {"question": RunnablePassthrough()} | prompt_template2 | llm | StrOutputParser()
chain_translate_output = (
    {"question": RunnablePassthrough()} | prompt_template3 | llm | StrOutputParser()
)

chain = (
    chain_translate_input
    | {"question": RunnablePassthrough(), "context": retriever}
    | prompt
    | llm
    | StrOutputParser()
    | chain_translate_output
)

In [ ]:
chain.invoke("O Jon Jones venceu a luta contra Stepe Miocic?")